# First/Every-visit MC prediction 


 
(c) Fabrice Mulotti

In [ ]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict

Create a blackjack environment:

In [ ]:
env = gym.make('Blackjack-v1')

## Définissons une politque

state[0] renvoie le nombre de point du joueur.   

L'action :      
0 est standby   
1 est hit (nouvelle carte)  

In [ ]:
def policy(pstate):
    # une ligne : définir une politique qui choisie l'action stand (0) si le joueur à 19 points ou moins, sinon hit (1)
    # le nombre de points est à l'indice

In [ ]:
# test !
assert policy([19,0,0])==1

In [ ]:
# test !
assert policy([20,0,0])==0

In [ ]:
## créons notre environnement
state = env.reset()
print(state)

In [ ]:
print(policy(state[0]))

***
## Generatrice d'épisode
Notre prochain fonction va nous permettre de générer des épisodes


In [ ]:
def generate_episode(policy,max_turn):
    
    # Enregistrement des épisodes
    episode = []
    
    # Remise à zéro env
    ge_state = env.reset()[0]
  
    # déroulement de la partie
    for i in range(max_turn):
        # choix d'une action en fonction de notre police et l'étar
        # 1 ligne
        
        # on joue notre action en on récupère les résultats
        ge_next_state, reward, done, truncated, info = # execution
        
        # enregistrement
        episode.append((ge_state, action, reward))
        
        # Condition de fin ?
        if done or truncated:
            break

        # sinon la partie continue
        ge_state = ge_next_state

    return episode

In [ ]:
# Devrait produit une partie
generate_episode(policy,100)

***
## Calcul de la fonction de valeur

Nous allons devoir générer un grand nombre d'épisode et ajuster la `valeur moyenne de v` à chaque passage sur un état s

 

In [ ]:
# total_return = fonction de valeur pour chaque état
total_return = defaultdict(float)

# N nombre de passage sur chaque état
N = defaultdict(int)

In [ ]:
# Nombre d'itération
num_iterations = 50000
gamma=0.90

In [ ]:
for i in range(num_iterations):
    
    # générons un épisode
    # générer un episode et sauvegarder dans la variable 'episode'

    # stockons l'état, action et récompense obtenue / zip permet de prendre chaque 1, 2eme, 3eme.. terme
    states, actions, rewards = zip(*episode)
    # pour chaque état rencontré dans l'épisode

    # calculer pour chaque état traversé, le gain G de l'état et mettre à jour les variables total_return[s] et N[s]
    R=0
    for t in range(len(states)-1,-1,-1):
        # 4-6 lignes
            

***
## Analysons nos résultats
Convertissons les données en dataframe pour une lecture plus facile

In [ ]:
total_return = pd.DataFrame(total_return.items(),columns=['state', 'total_return'])

In [ ]:
N = pd.DataFrame(N.items(),columns=['state', 'N'])

Merge des deux dataframe

In [ ]:
df = pd.merge(total_return, N, on="state")

Résultat : 

In [ ]:
df.head(10)

Calculons la fonction de valeur.
A ce stade nous avons tous les éléments (cumul des G et nombre de passage N)

In [ ]:
df['value'] = df['total_return']/df['N']

In [ ]:
df.head(20)

***
## Analyse

A ce stade nous pouvons apprécier la valeur de notre politique (seuil à 19).<br>
Exemple dans une situation ou nous avons 21 points.



In [ ]:
# sans AS
df[df['state']==(21,10,False)]['value'].values

In [ ]:
# avec AS
df[df['state']==(21,10,True)]['value'].values

La fonction de valeur est proche de 1. Donc efficiente.



---
Regardons maintenant dans un contexte plus difficile avec une main prochain de 19 (voir politique)

In [ ]:
df[df['state']==(19,8,False)]['value'].values

In [ ]:
df[df['state']==(19,8,True)]['value'].values

La valeur est moindre car l'incertitude de la suite de la partie plus grande